## Table of Contents
1. Importing Libraries
2. Loading Data
3. Data Preprocessing
4. Save Data

## 1. Imports and Settings

In [1]:

import pandas as pd
import numpy as np
import nltk

import os
import warnings
import re
import pickle
warnings.filterwarnings('ignore')



## 2. Load Data

In [2]:

# Current working directory
cwd='DataSet/BBC News Summary/'

# Path to the files
article_path=cwd+'News Articles/'
summary_path=cwd+'Summaries/'

# List of folders in the path
article_folders=os.listdir(article_path)
summary_folders=os.listdir(summary_path)

# List of files in the path
article_files=[]
summary_files=[]

for folder in article_folders:
    # append name with folder name
    article_files.append([folder+'/'+file for file in os.listdir(article_path+folder)])


for folder in summary_folders:
    # append name with folder name
    summary_files.append([folder+'/'+file for file in os.listdir(summary_path+folder)])


# Flatten the list
article_files=[item for sublist in article_files for item in sublist]
summary_files=[item for sublist in summary_files for item in sublist]


# Sort the list
article_files.sort()
summary_files.sort()


In [3]:

# Path to the files
article_files=[article_path+file for file in article_files]
summary_files=[summary_path+file for file in summary_files]

# Sample files
article_files[:5]

['DataSet/BBC News Summary/News Articles/business/001.txt',
 'DataSet/BBC News Summary/News Articles/business/002.txt',
 'DataSet/BBC News Summary/News Articles/business/003.txt',
 'DataSet/BBC News Summary/News Articles/business/004.txt',
 'DataSet/BBC News Summary/News Articles/business/005.txt']

In [4]:

# Sample files
summary_files[:5]

['DataSet/BBC News Summary/Summaries/business/001.txt',
 'DataSet/BBC News Summary/Summaries/business/002.txt',
 'DataSet/BBC News Summary/Summaries/business/003.txt',
 'DataSet/BBC News Summary/Summaries/business/004.txt',
 'DataSet/BBC News Summary/Summaries/business/005.txt']

In [5]:
# Read the first article and summary

with open(article_files[0], 'r') as file:
    article = file.read().replace('\n', '')


with open(summary_files[0], 'r') as file:
    summary = file.read().replace('\n', '')

print("Start")
print(article)
print("End")
print(summary)

Start
Ad sales boost Time Warner profitQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL

In [6]:

# Read all the articles and summaries and store them in a dataframe along with news category


# Create empty lists
articles=[]
summaries=[]
categories=[]
category=[]

# Read the files
for file in article_files:
    with open(file, 'r',encoding="utf8", errors='ignore') as f:
        articles.append(f.read().replace('\n', ''))
        category.append(file.split('/')[3])

for file in summary_files:
    with open(file, 'r',encoding="utf8", errors='ignore') as f:
        summaries.append(f.read().replace('\n', ''))

# Create a dataframe

df=pd.DataFrame({'Article':articles,'Summary':summaries,'Category':category})
df.head()




Article  \
0  Ad sales boost Time Warner profitQuarterly pro...   
1  Dollar gains on Greenspan speechThe dollar has...   
2  Yukos unit buyer faces loan claimThe owners of...   
3  High fuel prices hit BA's profitsBritish Airwa...   
4  Pernod takeover talk lifts DomecqShares in UK ...   

                                             Summary  Category  
0  TimeWarner said fourth quarter sales rose 2% t...  business  
1  The dollar has hit its highest level against t...  business  
2  Yukos' owner Menatep Group says it will ask Ro...  business  
3  Rod Eddington, BA's chief executive, said the ...  business  
4  Pernod has reduced the debt it took on to fund...  business

## 3. Data Preprocessing

In [7]:
contractions_dictionary = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}
                           


In [8]:

# Our Custom Function to clean the text


def clean_text(text):

    # Lower case
    text=text.lower()

    # Expand the words
    text=' '.join([contractions_dictionary[i] if i in contractions_dictionary.keys() else i for i in text.split()])

    # Remove punctuations
    text=re.sub(r'[^\w\s]','',text)

    # Remove numbers
    text=re.sub(r'\d+','',text)

    # Remove extra spaces
    text=re.sub(r'\s+',' ',text)

    # Remove extra spaces from start and end
    text=text.strip()


    return text


In [9]:
df.Article=df.Article.apply(clean_text)
df.Summary=df.Summary.apply(clean_text)

df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

df['Article_Length']=df.Article.apply(lambda x:len(x.split()))
df['Summary_Length']=df.Summary.apply(lambda x:len(x.split()))


df.head()

Article  \
0  ad sales boost time warner profitquarterly pro...   
1  dollar gains on greenspan speechthe dollar has...   
2  yukos unit buyer faces loan claimthe owners of...   
3  high fuel prices hit bas profitsbritish airway...   
4  pernod takeover talk lifts domecqshares in uk ...   

                                             Summary  Category  \
0  timewarner said fourth quarter sales rose to b...  business   
1  the dollar has hit its highest level against t...  business   
2  yukos owner menatep group says it will ask ros...  business   
3  rod eddington bas chief executive said the res...  business   
4  pernod has reduced the debt it took on to fund...  business   

   Article_Length  Summary_Length  
0             403             125  
1             377             159  
2             261             121  
3             376             186  
4             254             100

## 4. Save Data

In [10]:

# Save the dataframe into a csv file
df.to_csv('Resources/BBC_News_Summary.csv',index=False)
